In [3]:
# Import our dependencies
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [4]:
mlb_df = pd.read_csv("Resources/mlb_complete_8years.csv")
mlb_df.drop(columns=['Unnamed: 0','W','L','W-L%'], inplace=True)
mlb_df.head()

,Tm,BatAge,R/G,AB,R_x,H_x,2B,3B,HR_x,RBI,SB,CS,BB_x,SO_x,HBP_x,SH,SF,LOB_x,PAge,RA/G,tSho,SV,H_y,R_y,HR_y,BB_y,IBB,SO_y,HBP_y,BF,LOB_y,DefEff,Ch,PO,A,E,DP,post_season,Year
0,Arizona Diamondbacks_2022,26.5,4.33,5351,702,1232,262,24,173,658,104,29,531,1341,60,31,50,1039,30.0,4.57,10,33,1345,740,191,504,18,1216,59,6065,1051,0.704,5746,4290,1370,86,134,False,2022
1,Atlanta Braves_2022,27.5,4.87,5509,789,1394,298,11,243,753,87,31,470,1498,66,1,36,1030,30.0,3.76,9,55,1224,609,148,500,21,1554,62,6031,1101,0.701,5803,4344,1382,77,110,True,2022
2,Baltimore Orioles_2022,27.0,4.16,5429,674,1281,275,25,171,639,95,31,476,1390,83,12,43,1095,27.7,4.25,15,46,1406,688,171,443,8,1214,64,6058,1092,0.690,5920,4300,1529,91,151,False,2022
3,Boston Red Sox_2022,28.8,4.54,5539,735,1427,352,12,155,704,52,20,478,1373,63,12,50,1133,30.2,4.86,10,39,1411,787,185,526,17,1346,72,6167,1109,0.683,5825,4293,1447,85,134,False,2022
4,Chicago Cubs_2022,27.9,4.06,5425,657,1293,265,31,159,620,111,37,507,1448,84,19,36,1100,29.5,4.51,11,44,1342,731,207,540,19,1383,73,6162,1130,0.697,5880,4331,1453,96,139,False,2022


In [5]:
features_df = mlb_df.drop(columns=['Tm','post_season','Year'])
target_df = mlb_df['post_season'].astype(int)

In [37]:
target_df

0      0
1      1
2      0
3      0
4      0
      ..
234    1
235    0
236    1
237    1
238    0
Name: post_season, Length: 239, dtype: int32

In [6]:
features_df.shape

(239, 36)

In [7]:
features_df

,BatAge,R/G,AB,R_x,H_x,2B,3B,HR_x,RBI,SB,CS,BB_x,SO_x,HBP_x,SH,SF,LOB_x,PAge,RA/G,tSho,SV,H_y,R_y,HR_y,BB_y,IBB,SO_y,HBP_y,BF,LOB_y,DefEff,Ch,PO,A,E,DP
0,26.5,4.33,5351,702,1232,262,24,173,658,104,29,531,1341,60,31,50,1039,30.0,4.57,10,33,1345,740,191,504,18,1216,59,6065,1051,0.704,5746,4290,1370,86,134
1,27.5,4.87,5509,789,1394,298,11,243,753,87,31,470,1498,66,1,36,1030,30.0,3.76,9,55,1224,609,148,500,21,1554,62,6031,1101,0.701,5803,4344,1382,77,110
2,27.0,4.16,5429,674,1281,275,25,171,639,95,31,476,1390,83,12,43,1095,27.7,4.25,15,46,1406,688,171,443,8,1214,64,6058,1092,0.690,5920,4300,1529,91,151
3,28.8,4.54,5539,735,1427,352,12,155,704,52,20,478,1373,63,12,50,1133,30.2,4.86,10,39,1411,787,185,526,17,1346,72,6167,1109,0.683,5825,4293,1447,85,134
4,27.9,4.06,5425,657,1293,265,31,159,620,111,37,507,1448,84,19,36,1100,29.5,4.51,11,44,1342,731,207,540,19,1383,73,6162,1130,0.697,5880,4331,1453,96,139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,28.0,4.69,5498,759,1369,248,9,205,725,63,32,525,1380,80,46,48,1101,26.7,4.27,8,43,1354,691,144,593,49,1337,67,6271,1214,0.688,6107,4394,1617,96,159
235,27.1,4.42,5475,716,1415,274,43,150,664,128,51,540,1388,101,28,50,1156,27.1,3.99,14,52,1236,646,164,501,34,1421,53,5992,1001,0.701,5863,4360,1408,95,118
236,27.4,4.55,5453,737,1308,266,24,194,696,74,35,555,1484,88,33,34,1093,30.4,5.23,5,42,1516,848,222,491,23,1121,72,6231,1090,0.687,6142,4328,1695,119,169
237,28.9,4.38,5477,709,1336,320,16,217,680,47,30,499,1387,58,5,37,1036,29.3,5.14,3,39,1476,832,208,551,19,1298,67,6265,1132,0.708,6003,4323,1592,88,145


In [8]:
X_train, X_test, y_train, y_test = train_test_split(features_df, target_df, random_state=78)
X_scaler = skl.preprocessing.StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=36))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [10]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [11]:
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

INFO:tensorflow:Oracle triggered exit


In [12]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 3,
 'num_layers': 2,
 'units_0': 9,
 'units_1': 7,
 'units_2': 5,
 'units_3': 7,
 'units_4': 1,
 'units_5': 3,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [13]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 0.4786 - accuracy: 0.8333 - 225ms/epoch - 112ms/step
Loss: 0.478573739528656, Accuracy: 0.8333333134651184


In [14]:
best_model

In [35]:
from keras.models import load_model
best_model.save('mlb_network.h5')
mlb_model = load_model("mlb_network.h5")
loss, accuracy = mlb_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 5ms/step - loss: 2100.2017 - accuracy: 0.7000


In [17]:
bat_23 = pd.read_csv("Resources/2023_Batting.csv")
bat_23.drop(columns=['#Bat','G','PA','BA','OBP','SLG', 'OPS', 'OPS+','TB', 'GDP','IBB'], inplace=True)
pit_23 = pd.read_csv("Resources/2023_Pitching.csv")
pit_23.drop(columns=['#P', 'ERA', 'G', 'GS', 'GF','CG', 'cSho', 'IP', 'ER', 'BK', 'WP','ERA+', 'FIP', 'WHIP', 'H9', 'HR9','BB9', 'SO9', 'SO/W'], inplace=True)
def_23 = pd.read_csv("Resources/2023_Fielding.csv")
def_23.drop(columns=['#Fld', 'RA/G', 'G', 'GS', 'CG', 'Inn', 'Fld%', 'Rtot', 'Rtot/yr', 'Rdrs', 'Rdrs/yr', 'Rgood'], inplace=True)
merged_23 = pd.merge(bat_23, pit_23, on='Tm')
merged_23 = pd.merge(merged_23, def_23, on = 'Tm')
merged_23.head()

,Tm,BatAge,R/G,AB,R_x,H_x,2B,3B,HR_x,RBI,SB,CS,BB_x,SO_x,HBP_x,SH,SF,LOB_x,PAge,RA/G,W,L,W-L%,tSho,SV,H_y,R_y,HR_y,BB_y,IBB,SO_y,HBP_y,BF,LOB_y,DefEff,Ch,PO,A,E,DP
0,Arizona Diamondbacks,27.4,5.02,1864,276,487,107,12,64,262,51,8,166,414,16,11,18,357,28.6,4.71,32,23,0.582,4,15,460,259,63,197,4,463,13,2083,363,0.697,1985,1465,501,19,38
1,Atlanta Braves,27.9,5.05,1861,278,478,98,4,91,269,36,8,181,478,23,0,11,357,30.2,4.04,32,23,0.582,5,13,449,222,51,176,4,524,25,2064,385,0.673,1951,1461,454,36,48
2,Baltimore Orioles,27.2,4.96,1850,273,464,100,10,64,265,45,10,199,448,15,9,18,364,28.4,4.38,35,20,0.636,4,18,474,241,64,177,9,496,17,2099,377,0.683,1992,1490,478,24,61
3,Boston Red Sox,28.6,5.20,1851,281,489,120,4,61,271,31,6,175,431,24,5,11,365,31.0,5.02,28,26,0.519,0,14,465,271,76,162,2,450,26,2047,356,0.691,1922,1422,466,34,37
4,Chicago Cubs,28.7,4.54,1837,245,468,82,7,66,237,41,12,207,497,17,2,15,409,29.9,4.43,24,30,0.444,6,7,444,239,51,163,3,453,20,2006,339,0.690,1956,1434,497,25,55


In [19]:
merged_23.to_csv('Resources/season2023.csv')

In [27]:
season_2023 = pd.read_csv('Resources/season2023.csv')
season_2023.drop(columns=['Unnamed: 0','W','L','W-L%'], inplace=True)
season_2023.set_index('Tm', inplace=True)

In [28]:
season_2023.head(3)

,BatAge,R/G,AB,R_x,H_x,2B,3B,HR_x,RBI,SB,CS,BB_x,SO_x,HBP_x,SH,SF,LOB_x,PAge,RA/G,tSho,SV,H_y,R_y,HR_y,BB_y,IBB,SO_y,HBP_y,BF,LOB_y,DefEff,Ch,PO,A,E,DP
Tm,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Arizona Diamondbacks,27.4,5.02,1864,276,487,107,12,64,262,51,8,166,414,16,11,18,357,28.6,4.71,4,15,460,259,63,197,4,463,13,2083,363,0.697,1985,1465,501,19,38
Atlanta Braves,27.9,5.05,1861,278,478,98,4,91,269,36,8,181,478,23,0,11,357,30.2,4.04,5,13,449,222,51,176,4,524,25,2064,385,0.673,1951,1461,454,36,48
Baltimore Orioles,27.2,4.96,1850,273,464,100,10,64,265,45,10,199,448,15,9,18,364,28.4,4.38,4,18,474,241,64,177,9,496,17,2099,377,0.683,1992,1490,478,24,61


In [29]:
season_2023.shape

(30, 36)

In [31]:
scaled_2023 = X_scaler.transform(season_2023)

In [44]:
predictions = best_model.predict(scaled_2023)

1/1 [==============================] - 0s 43ms/step


In [46]:
predictions

array([[0.5197171 ],
       [0.48802292],
       [0.52494043],
       [0.30518097],
       [0.5282531 ],
       [0.33049268],
       [0.35777876],
       [0.43210676],
       [0.34167314],
       [0.35054088],
       [0.5282531 ],
       [0.29382327],
       [0.46787795],
       [0.5282531 ],
       [0.42987847],
       [0.45540187],
       [0.5282531 ],
       [0.4074618 ],
       [0.5282531 ],
       [0.07608607],
       [0.29674834],
       [0.50572634],
       [0.45453715],
       [0.5282531 ],
       [0.40275803],
       [0.4149126 ],
       [0.5121684 ],
       [0.5282531 ],
       [0.5151501 ],
       [0.3257315 ]], dtype=float32)

In [57]:
season_2023['predictions'] = predictions
season_2023_predict = season_2023.copy()

In [59]:
season_2023_predict = season_2023_predict[['predictions']]

In [65]:
season_2023_predict.sort_values(by=['predictions'], ascending=False,inplace=True)
season_2023_predict.head(10)


,predictions
Tm,
Chicago Cubs,0.528253
Texas Rangers,0.528253
Seattle Mariners,0.528253
New York Yankees,0.528253
Houston Astros,0.528253
Minnesota Twins,0.528253
Los Angeles Dodgers,0.528253
Baltimore Orioles,0.524940
Arizona Diamondbacks,0.519717
